# Libraries

In [1]:
from mnist import MNIST

# Loading and Organinzing the Data

In [2]:
mndata = MNIST('./Data')
images, lables = mndata.load_training()
lables = lables.tolist()

In [3]:
# Oraganize the data
images = [x for _,x in sorted(zip(labels,images))]
labels.sort()

NameError: name 'labels' is not defined